In [73]:
import os
import fnmatch
import numpy as np
import nibabel as nb
from nipype.interfaces import afni
from nipype.utils.filemanip import split_filename
#fs=1./6.9698 #1/TR
fs=1./23.2 #1/DynScanTime (résolution temporelle different du TR car pas comme en EPI)

p='/NAS/dumbo/protocoles/CAR-T-Cell/process/DERYCKE_CORINNEE_fsl_mc/rdyn.nii.gz'
path, name, ext = split_filename(p)
out_file = os.path.join(path, 'f' + name + ext)
img = nb.load(p)

In [78]:
highpass_freq = 0.0000000001
lowpass_freq = 0.01
timepoints = img.shape[-1]
F = np.zeros((timepoints))
lowidx = timepoints/2 + 1
print(lowidx)
if lowpass_freq > 0:
    lowidx = np.round(float(lowpass_freq) / fs * timepoints)
print(lowidx)
highidx = 0
if highpass_freq > 0:
    highidx = np.round(float(highpass_freq) / fs * timepoints)
F[int(highidx):int(lowidx)] = 1
F = ((F + F[::-1]) > 0).astype(int)
print(lowidx)
print(highidx)
print(F)

16.0
7.0
7.0
0.0
[1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1]


In [71]:
data = img.get_data()
print('start filtering')
if np.all(F == 1):
    filtered_data = data
else:
    filtered_data = np.real(np.fft.ifftn(np.fft.fftn(data) * F))+np.repeat(np.mean(data, (3))[:, :,:, np.newaxis], data.shape[3], axis=3)
print('finish filtering')

start filtering
finish filtering


In [72]:
img_out = nb.Nifti1Image(filtered_data, img.affine,img.header)
img_out.to_filename(out_file)
print('saved %s'%(out_file))

saved /NAS/dumbo/protocoles/CAR-T-Cell/process/DERYCKE_CORINNEE_fsl_mc/frdyn.nii.gz
